Sara Frusone




# LEARNING DYNAICAL SYSTEMS WITH KERNELS
# Logistic Map Notebook

Predicting values generating logistic maps with different non-periodicity/periodicity.

### Logistic map theory overview
The logistic map was derived from a differential equation describing population growth, popularized by Robert May. The dynamical equation is as follows:

$x_{n+1}=rx_{n}(1−x_{n})$

where $r$ can be considered akin to a growth rate, $x_{n+1}$ is the population next year, and $x_{n}$ is the current population. Population ranges between 0 and 1, and signifies the proportion of the maximum population. What happens to population over time at a fixed $r$ value?


As $r$ increases, the periodicity increases until it becomes infinite, and infinite periodicity is equivalent to aperiodicity. This occurs via period doubling: as can be seen clearly from the logistic map, one period splits into two, which split into four, which split into eight etc. This period doubling occurs with less and less increase in r such that an infinite period is reached within a finite increase in $r$, and at this point the map is aperiodic. This occurs whenever there is a transition from periodicity to aperiodicity.


Importing all libraries 

In [ ]:

import numpy as np 
from numpy import arange
from numpy import mean
from numpy import std
from numpy import absolute

import pandas as pd  
from pandas import read_csv

import matplotlib.pyplot as plt
from matplotlib import pyplot as plt 
plt.rcParams.update({'font.size': 22}) #enlarge plot font
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 4
mpl.rcParams['legend.fontsize'] = 15
mpl.rcParams['xtick.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 15
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import matplotlib.pylab as pl
from decimal import Decimal

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

from sklearn.gaussian_process.kernels import ExpSineSquared, RationalQuadratic ,Matern ,RBF, ConstantKernel, DotProduct, WhiteKernel
# https://scikit-learn.org/stable/modules/gaussian_process.html
from sklearn.kernel_ridge import KernelRidge

from scipy import stats

# RIDGE REGRESSION
Let us consider the dataset generated by the logistic map implemented above, in the different setting (stationary and not stationary).
Let us generate the dataset and plotting it in order to check the stationarity.

In [ ]:
def Dataset_generator(r, steps):
    # initialize an array of 0s and specify starting values and r constant
    x = np.zeros(steps + 1)
    y = np.zeros(steps + 1)
    x[0], y[0] = 0, 0.4
    dataset= []
    dataset_y= []


    # loop over the steps and replace array values with calculations
    for i in range(steps):
        y[i+1] = r * y[i] * (1 - y[i])
        dataset.append((y[i],y[i+1])) #points should be (y_1, y_2), (y_2,y_3)
        dataset_y.append(y[i+1])
    # print(len(dataset_y))
    # print(len(dataset))

    
    
    return dataset, dataset_y

Two functions are implemented to plot the dataset (they differ for the grid representation)

In [ ]:
def DatasetPlotter1(r, steps, dataset, dataset_y):
# plot the figure!
    fig, ax = plt.subplots(1,2, figsize=(25,5))
    
    ax[0].plot(dataset_y,  alpha=0.5)
    ax[0].set(xlabel='Index', ylabel='Function value')
    ax[0].set_title('Dataset plot with r: %s' % r)
    ax[0].grid(visible=True, axis='both')
    
        
            
    # plot the figure(y_1, y_2), (y_2,y_3)
    flat_dataset_X = np.array([elem[0] for elem in dataset])[:,None]
    flat_dataset_y= np.array([elem[1] for elem in dataset])[:,None] #dataset_y
    
    ax[1].set(xlabel='y [i]', ylabel='y [i+1]')
   
    ax[1].set_yscale('log')
    ax[1].set_xscale('log')
    #ax[1].ticklabel_format(axis='y', style='sci',scilimits=(0,0))
    ax[1].set_title('Dataset plot with r: %s' % r)
    ax[1].grid(visible=True,which='both', axis='both')
    
    l1= ax[1].scatter(flat_dataset_X,flat_dataset_y)
    ax[1].legend([l1],["datset"], loc= "best")
    plt.show()  

In [ ]:
def DatasetPlotter2(r, steps, dataset, dataset_y):
# plot the figure!
    fig, ax = plt.subplots(1,2, figsize=(25,5))
    
    ax[0].plot(dataset_y,  alpha=0.5)
    ax[0].set(xlabel='Index', ylabel='Function value')
    ax[0].set_title('Dataset plot with r: %s' % r)
    ax[0].grid()
     
     
            
    # plot the figure(y_1, y_2), (y_2,y_3)
    flat_dataset_X = np.array([elem[0] for elem in dataset])[:,None]
    flat_dataset_y= np.array([elem[1] for elem in dataset])[:,None] #dataset_y
    
    ax[1].set(xlabel='y [i]', ylabel='y [i+1]')
   
    ax[1].set_yscale('log')
    ax[1].set_xscale('log')
    ax[1].set_title('Dataset plot with r: %s' % r)
    ax[1].grid()
    
    l1= ax[1].scatter(flat_dataset_X,flat_dataset_y) 
    ax[1].legend([l1],["datset"], loc= "best")
    plt.show()  

The Mean Absolute Percentage Error (MAPE) is one of the most commonly used KPIs to measure forecast accuracy.

MAPE is the sum of the individual absolute errors divided by the demand (each period separately). It is the average of the percentage errors.
$$
\mathrm{MAPE}=\frac{100 \%}{n} \sum_{i=1}^n\left|\frac{Y_i-\hat{Y}_i}{Y_i}\right|
$$
##### Remark
MAPE can be problematic. Most pointedly, it can cause division-by-zero errors. My guess is that this is why it is not included in the sklearn metrics.

In [ ]:
#Defining MAPE function
def MAPE(Y_true,Y_Predicted):
    mape = np.mean(np.abs((Y_true - Y_Predicted)/Y_true))*100
    return mape

#### MAD error
Mean absolute deviation (MAD) is another commonly used forecasting metric. This metric shows how large an error, on average, you have in your forecast. However, as the MAD metric gives you the average error in units, it is not very useful for comparisons.

$$
\text { MAD }=\frac{1}{n} \sum_{i=1}^n \mid Y_i-\hat{Y}_i\mid
$$

### MSE

The MSE either assesses the quality of a predictor (i.e., a function mapping arbitrary inputs to a sample of values of some random variable), or of an estimator MSE differs according to whether one is describing a predictor or an estimator.
Predictor with $\hat{Y}$ being the predicted values (e.g. as from a least-squares fit), then the within-sample MSE of the predictor is computed as
$$
\mathrm{MSE}=\frac{1}{n} \sum_{i=1}^n\left(Y_i-\hat{Y}_i\right)^2
$$
In other words, the MSE is the mean $\left(\frac{1}{n} \sum_{i=1}^n\right)$ of the squares of the errors $\left(Y_i-\hat{Y}_i\right)^2$. This is an easily computable quantity for a particular sample (and hence is sample-dependent).


It is provided by the library directly.


#### Splitting 

Now we try different splitting of the dataset in train and test such as 50/50 and 80/20, 90/10 

In [ ]:
#splitting function that given data, splits them according to the settled percentage pecentages, 
def split(X,y, test_size): #from 0 to 1
    train_size= 1-test_size
    X_train= X[:int(len(X)*train_size)]
    X_test= X[int(len(X)*train_size):]
    y_train= y[:int(len(y)*train_size)]
    y_test= y[int(len(X)*train_size):]  
    return X_train, X_test, y_train, y_test  

Let us apply the ridge regression model to the dataset
Let us plot the real value and the the predicted 

It is very important the way we select the kernels since it strongly affects the forecast
manually select the best parameter: Select an appropriate value for alpha

In [ ]:
#define a function that acts ridge regression, given a kernel alpha and splitted data 

def RIDGE(X_train, X_test, y_train, y_test,Kernel, alpha):
    
    print('Selected kernel: ' +str(Kernel))

    #model
    model = KernelRidge(alpha,kernel=Kernel).fit(X_train, y_train) 
    #compute predictions
    pred_train= model.predict(X_train)
    pred_test= model.predict(X_test) 
   
    #train
    mape_train = MAPE(y_train, pred_train)
    mse_train = mean_squared_error(pred_train,y_train)
    print('Mean square error train: %.5f' % mse_train )
    print('MAPE train: %.5f' % mape_train)


    #predictions test
    mape_test = MAPE(y_test, pred_test)
    mse_test = mean_squared_error(pred_test,y_test)
    print('Mean square error test: %.5f' %mse_test )
    print('MAPE test: %.5f' % mape_test)
    #print('Maximum error test: %.3f' % np.max(np.linalg.norm(model.predict(X_test)-y_test)))
    
    return mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test

In [ ]:
#define a function that acts the gaussian process regressor
# it returns mape, mse error on train and test 

def GAUS(X_train, X_test, y_train, y_test, alpha):
    
    #second model
    kernel = DotProduct() + WhiteKernel()
    model = GaussianProcessRegressor(kernel=kernel, alpha=alpha, random_state=0).fit(X_train, y_train)
      
    #compute predictions
    pred_train= model.predict(X_train)
    pred_test= model.predict(X_test)
    
    
    #train
    mape_train = MAPE(y_train, pred_train)
    mse_train = mean_squared_error(pred_train,y_train)
    
    print('Mean square error train: %.5f' % mse_train )
    print('MAPE train: %.5f' % mape_train)

    #predictions test
    mape_test = MAPE(y_test,pred_test)
    mse_test = mean_squared_error(pred_test,y_test)
    
    print('Mean square error test: %.5f' %mse_test )
    print('MAPE test: %.5f' % mape_test)
   #print('Maximum error test: %.3f' % np.max(np.linalg.norm(model.predict(X_test)-y_test)))
    
    return mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test

The following functions differ for the grid shown in the plot. We will use them to plot the scatter of $(y[i], y[i+1])$

In [ ]:
def plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test):
    fig, ax = plt.subplots(1,3, figsize=(20,5))
    
    #train and test
    
    ax[0].set(xlabel='y [i]', ylabel='y [i+1]')
    ax[0].set_yscale('log')
    ax[0].set_xscale('log')
    #ax[0].ticklabel_format(axis='both', style='sci',scilimits=(0,0))
    
    l1= ax[0].scatter(X_train,y_train, alpha=0.5, color= 'b')
    l2=ax[0].scatter(X_train,pred_train, alpha=0.5, color='orange')
    l3= ax[0].scatter(X_test, y_test, alpha=0.5, color= 'red')
    l4=ax[0].scatter(X_test, pred_test, alpha=0.5, color='green')
    ax[0].set_title('Train and test predictions with r: %s' % r)
    ax[0].legend([l1,l2,l3, l4],["train real", "train predicted", "test real", "test predicted"], loc= "best")
    ax[0].grid(visible=True,which='both', axis='both')
    
    #train
    
    ax[1].set(xlabel='y [i]', ylabel='y [i+1]')
    ax[1].set_yscale('log')
    ax[1].set_xscale('log')
    #ax[1].ticklabel_format(axis='both', style='sci',scilimits=(0,0))
    
    ax[1].grid(visible=True,which='both', axis='both')
    l1= ax[1].scatter(X_train,y_train, alpha=0.5, color= 'b')
    l2=ax[1].scatter(X_train,pred_train, alpha=0.5, color='orange')
    ax[1].set_title('Train predictions with r: %s' % r)
    ax[1].legend([l1, l2],["real", "predicted"], loc= "best")

        
    #test
    
    ax[2].set(xlabel='y [i]', ylabel='y [i+1]')
    ax[2].set_yscale('log')
    ax[2].set_xscale('log')
    
    #ax[2].ticklabel_format(axis='both', style='sci',scilimits=(0,0))
    ax[2].grid(visible=True,which='both', axis='both')
    l3= ax[2].scatter(X_test, y_test, alpha=0.5, color= 'red')
    l4=ax[2].scatter(X_test, pred_test, alpha=0.5, color='green')
    ax[2].set_title('Test predictions with r: %s' % r)
    ax[2].legend([l3, l4],["real", "predicted"], loc= "best")

    
    plt.tight_layout()
    plt.show()    

In [ ]:
def plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test):

    fig, ax = plt.subplots(1,3, figsize=(20,5))
    
    #train and test

    ax[0].set(xlabel='y [i]', ylabel='y [i+1]')
    ax[0].set_yscale('log') 
    ax[0].set_xscale('log')
    
    #ax[0].ticklabel_format(axis='both', style='sci',scilimits=(0,0))
    l1= ax[0].scatter(X_train,y_train, alpha=0.5, color= 'b')
    l2=ax[0].scatter(X_train,pred_train, alpha=0.5, color='orange')
    l3= ax[0].scatter(X_test, y_test, alpha=0.5, color= 'red')
    l4=ax[0].scatter(X_test, pred_test, alpha=0.5, color='green')
    ax[0].set_title('Train and test predictions with r: %s' % r)
    ax[0].legend([l1,l2,l3, l4],["train real", "train predicted", "test real", "test predicted"], loc= "best")
    ax[0].grid()
    
    #train
    
    ax[1].set(xlabel='y [i]', ylabel='y [i+1]')
    ax[1].set_yscale('log')
    ax[1].set_xscale('log')
    
    #ax[1].ticklabel_format(axis='both', style='sci',scilimits=(0,0))
    ax[1].grid()
    l1= ax[1].scatter(X_train,y_train, alpha=0.5, color= 'b')
    l2=ax[1].scatter(X_train,pred_train, alpha=0.5, color='orange')
    ax[1].set_title('Train predictions with r: %s' % r)
    ax[1].legend([l1, l2],["real", "predicted"], loc= "best")

        
    #test
    
    ax[2].set(xlabel='y [i]', ylabel='y [i+1]')
    ax[2].set_yscale('log')
    ax[2].set_xscale('log')
    
    #ax[2].ticklabel_format(axis='both', style='sci',scilimits=(0,0))
    ax[2].grid()
    l3= ax[2].scatter(X_test, y_test, alpha=0.5, color= 'red')
    l4=ax[2].scatter(X_test, pred_test, alpha=0.5, color='green')
    ax[2].set_title('Test predictions with r: %s' % r)
    ax[2].legend([l3, l4],["real", "predicted"], loc= "best")

    
    plt.tight_layout()
    plt.show()  

The following functions plot the value corresponding to each index $i$, which is the couple $(i,y[i])$. They differ for the grid representation in the plot.

In [ ]:
def plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test):
    
# plot the figure

    # Create 2x2 sub plots
    gs = gridspec.GridSpec(2, 2)

    ax =pl.figure(figsize=(20,18))
    
    
    ax = pl.subplot(gs[0, :]) # row 0, span all
       
    ax.grid(visible=True,which='both', axis='both')
    ax.set_yscale('log')
    l1,=ax.plot(y_train, alpha=0.5, color='b')
    l2,=ax.plot(pred_train, alpha=0.5, color='orange')
    l3,=ax.plot(np.arange((steps-steps*test_size), steps),y_test, alpha=0.5, color= 'red') 
    l4,=ax.plot(np.arange((steps-steps*test_size), steps),pred_test, alpha=0.5, color= 'green')
    ax.set(xlabel='Index', ylabel='Function value')
    ax.set_title('Real and predictions with r: %s' % r)
    ax.legend([l1, l2,l3,l4],["real train","predicted train" ,"real test", "predicted test"], loc= "best")
    

    ax= pl.subplot(gs[1, 0]) # row 1, col 0
       
    ax.grid(visible=True,which='both', axis='both')
    ax.set_yscale('log')
    l1,=ax.plot(y_train, alpha=0.5, color='b')
    l2,=ax.plot(pred_train, alpha=0.5, color='orange')
    ax.set(xlabel='Index', ylabel='Function value')
    ax.set_title('Train real and predictions with r: %s' % r)
    ax.legend([l1, l2],["real train","predicted train" ], loc= "best")
    
    ax = pl.subplot(gs[1, 1]) # row 1, col 1
       
    ax.grid(visible=True,which='both', axis='both')
    ax.set_yscale('log')
    l3,=ax.plot(np.arange((steps-steps*test_size), steps),y_test, alpha=0.5, color= 'red') 
    l4,=ax.plot(np.arange((steps-steps*test_size), steps),pred_test, alpha=0.5, color= 'green')
    ax.set(xlabel='Index', ylabel='Function value')
    ax.set_title('Test real and predictions with r: %s' % r)
    ax.legend([l3,l4],["real test", "predicted test"], loc= "best")
    
    plt.show()

In [ ]:
def plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test):
    
    # plot the figure

    # Create 2x2 sub plots
    gs = gridspec.GridSpec(2, 2)
    ax =pl.figure(figsize=(20,18))
    
    
    ax = pl.subplot(gs[0, :]) # row 0, span all
       
    ax.grid()
    ax.set_yscale('log')
    l1,=ax.plot(y_train, alpha=0.5, color='b')
    l2,=ax.plot(pred_train, alpha=0.5, color='orange')
    l3,=ax.plot(np.arange((steps-steps*test_size), steps),y_test, alpha=0.5, color= 'red') 
    l4,=ax.plot(np.arange((steps-steps*test_size), steps),pred_test, alpha=0.5, color= 'green')
    ax.set(xlabel='Index', ylabel='Function value')
    ax.set_title('Real and predictions with r: %s' % r)
    ax.legend([l1, l2,l3,l4],["real train","predicted train" ,"real test", "predicted test"], loc= "best")
    

    ax= pl.subplot(gs[1, 0]) # row 1, col 0
       
    ax.grid()
    ax.set_yscale('log')
    l1,=ax.plot(y_train, alpha=0.5, color='b')
    l2,=ax.plot(pred_train, alpha=0.5, color='orange')
    ax.set(xlabel='Index', ylabel='Function value')
    ax.set_title('Train real and predictions with r: %s' % r)
    ax.legend([l1, l2],["real train","predicted train" ], loc= "best")
    
    ax = pl.subplot(gs[1, 1]) # row 1, col 1
       
    ax.grid()
    ax.set_yscale('log')
    l3,=ax.plot(np.arange((steps-steps*test_size), steps),y_test, alpha=0.5, color= 'red') 
    l4,=ax.plot(np.arange((steps-steps*test_size), steps),pred_test, alpha=0.5, color= 'green')
    ax.set(xlabel='Index', ylabel='Function value')
    ax.set_title('Test real and predictions with r: %s' % r)
    ax.legend([l3,l4],["real test", "predicted test"], loc= "best")
    
    plt.show()

It can be interesting to zoom in the function representation, thanks to the zoom function mentioned below it is possible to zoom the plot and confront the prediction with the real valules graphically, in any interval of both axes.

It will be necessary to insert the intervals on both axes where we want to zoom in.

In [ ]:

def zoomplotter_steps(range_low_lim_x, range_up_lim_x,range_low_lim_y, range_up_lim_y,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test):
    # Create main container 
    fig, ax = plt.subplots(figsize=(20,5))
    plt.subplots_adjust(bottom = 0., left = 0, top = 1., right = 1)
    ax.set_yscale('log')
    ax.set_xlim(range_low_lim_x, range_up_lim_x)
    ax.set_ylim(range_low_lim_y, range_up_lim_y)
    l1,=ax.plot(y_train,alpha=0.5, color='b')#,"r--")
    l2,=ax.plot(pred_train,alpha=0.5, color='orange')
    l3,=ax.plot(np.arange((steps-steps*test_size), steps),y_test, alpha=0.5, color='red') 
    l4,=ax.plot(np.arange((steps-steps*test_size), steps),pred_test, alpha=0.5, color= 'green')
    ax.set(xlabel='Index', ylabel='Function value')
    ax.set_title('Zoom of the plot real and predictions with r: %s' % r)
    ax.legend([l1, l2,l3,l4],["real train","predicted train" ,"real test", "predicted test"], loc= "best")
    ax.grid()
    
    fig.tight_layout()
    plt.show()



This function shows how the size of the train and test (so their proportion) impacts on the predictions of the algorithm.

In [ ]:

# plot train size vs mse error

def trainVSmse(steps, sizes, mse_train_list, mse_test_list):
  fig, ax = plt.subplots(figsize=(7,7))
  array_sizes=np.array(sizes)*100
  l1,= ax.plot(array_sizes,mse_train_list)
  l2,=ax.plot(array_sizes, mse_test_list)
  ax.ticklabel_format(axis='y', style='sci',scilimits=(0,0))
  ax.set(xlabel='Dataset percentage of the test (%)', ylabel='MSE error')
  plt.title('Test size vs MSE')
  
  ax.legend([l1, l2],["MSE train","MSE test"], loc= "best")
  ax.grid(visible=True,which='both', axis='both')
  fig.tight_layout()
  plt.show()
  

When we tune the alpha parameter, thanks to this plot it is possible to see how the alpha value impacts on the MSE error.

In [ ]:
# plot alpha and MSE error

def alphaVSmse(alphas, mse_train_list, mse_test_list):   

  fig, ax = plt.subplots(figsize=(7,7))
  l1,=ax.plot(alphas, mse_train_list,alpha=0.5)
  l2,=ax.plot(alphas, mse_test_list,alpha=0.5)
  plt.legend([l1, l2],["MSE train", "MSE test"], loc= "best")
  ax.ticklabel_format(axis='y', style='sci',scilimits=(0,0))
  ax.set(xlabel='alpha', ylabel='MSE')
  plt.title('Alphas vs MSE')
  plt.grid(visible=True,which='both', axis='both')
  fig.tight_layout()
  plt.show()  

Building a data frame where we can insert the values of the experiments can be usefull to then confron all the results, and have a general overview.

What we would like to observe are: the $r$ value which impacts on the periodicity, $steps$ which is the number of points of the dataset generated, $test \_ size$ the way we splitted the original dataset, the $model$ we used (which is adapted to the periodicity of the generated function), $\alpha$ parameter, the errors o both training and set

In [ ]:

# Calling DataFrame constructor  
def DataFrameConstructor(r, steps, test_size,model, alpha,  mse_train, mse_test ):
## list of strings
    dict={'r':[r],'steps':[steps],'test_size':[test_size],'model':[model],'alpha':[alpha], 'mse_train':[mse_train],'mse_test': [mse_test] }  

    df = pd.DataFrame(dict)
    #display(df)
    return df 

It can be interesting to inject some noise and see how it can impact our predictions. To test the robustnesss of the models in the different situations (such as the different periodicity of the dataset, the different kernel used for the predictions).

In [ ]:
# add noise function

def noiseadder(steps, dataset_X, dataset_y):
    noise = np.random.normal(3,4,steps)

    dataset_y_noisy= dataset_y+ noise
    dataset_X_noisy=np.array([elem[0] +noise for elem in dataset_X])
    

    return dataset_X_noisy, dataset_y_noisy

Now we start the experiments.

First we initialize the data frame, where we will insert the experiments results.

In [ ]:
# initialize data frame
dict={'r':[],'steps':[],'test_size':[],'model':[],'alpha':[],  'mse_train':[],'mse_test': [] }  
#empty data frame:
df = pd.DataFrame(dict)

# Periodic and linear experiments


### $r=0.5$ and $50$ steps

Let us generate the dataset with r=0.5, and 50 steps.


When r is small (less than one, to be specific), the population heads towards 0. 

In [ ]:
r =0.5
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
DatasetPlotter1(r, steps, dataset_X, dataset_y)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]


In [ ]:
#parameters
alpha= 0.1
kernel= 'linear'
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 


for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)

    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-5,1e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)    


### Comments
When r is small (less than one, to be specific), the population heads towards 0. 

In [ ]:
# display the data frame    
#display(df)

we have been testing different kernels, but the linear one better suits the linearity of the function we have to predict, so we will avoid experimets with the other kernels sincethe results are worst.

As a reminder the tested kernels are the following:
- kernel=  1.0 * Matern(length_scale=1.0, nu=1.5)
- kernel= 1.0 * RBF(1.0)
The code is the same of the ablove box, but changing the kernel (If you wish to see the result, you can change it)

It is now interesting to see what happens changing the number of points of the dataset. Below the results generating a dataset of 1000 points.

### $r =0.5$ and $steps=1000$

In [ ]:
r =0.5
steps=1000
dataset_X, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:
DatasetPlotter2(r, steps, dataset_X, dataset_y)

In [ ]:

alpha= 0.1
kernel= 'linear'
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 

for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)

    
    
    dr= DataFrameConstructor(r, steps, test_size,model, alpha, mse_train, mse_test)
    df= df.append(dr)
    
    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps1(steps,r,test_size,X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-5,1e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)


trainVSmse(steps, sizes, mse_train_list, mse_test_list)    

In [ ]:
# display the data frame    

#display(df)

### tuning the parameter alpha
An other interesting point is to consider some regularization parameters to better understand in the linear example which is very simple, how selecting the proper parameter can impact on the forecasting accuracy (mse for example).

In [ ]:
r =0.5
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
DatasetPlotter1(r, steps, dataset_X, dataset_y)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:

alphas= np.arange(0.00001, 2, 0.01)
kernel= 'linear'
sizes= (0.1,0.2,0.3, 0.5)



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)

    #define one for each alpha
    mape_train_list=[]
    mape_test_list=[] 
    mse_train_list=[]
    mse_test_list=[] 

    #Select an appropriate value for alpha
    for alpha in alphas:
    
      mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
      

      
      dr= DataFrameConstructor(r, steps, test_size,model, alpha, mse_train, mse_test)
      df= df.append(dr)
      
      #save errors in arrays
      mape_train_list.append(mape_train)
      mape_test_list.append(mape_test)
      
      mse_train_list.append(mse_train)
      mse_test_list.append(mse_test)

      # plot the figure!
      # plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)

      # plotter_steps2(steps,r,test_size,X_train, X_test, y_train, y_test, pred_train, pred_test)

alphaVSmse(alphas, mse_train_list, mse_test_list)    

In [ ]:
# display the data frame    
display(df)

### $r=2.5$, $steps= 50$
As r is increased to 2.5, a stable population is reached

This is called a period one trajectory because it takes one iteration to return to the starting population size. Because period one occurs when $x_{n+1}=x_{n}$, we can algebraically determine the population values reached.

In [ ]:
r=2.5
steps=50
dataset, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[1] for elem in dataset])[:,None]

In [ ]:
    # plot the figure!
   
    l1= plt.scatter(flat_dataset_X,dataset_y, alpha=0.5)
    plt.xlabel('y [i]')
    plt.ylabel('y [i+1]')

    plt.legend([l1],["datset"], loc= "best")
    plt.title('Dataset plot with r: %s' % r)
    plt.grid()
    plt.show() 

In [ ]:

#alphas= np.arange(0.00001, 2, 0.01)
sizes=(0.1,0.2,0.3, 0.5)
kernel='linear'
alpha=0.1

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 

for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    kernel=DotProduct() + WhiteKernel() #TBD
    



    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test, kernel, alpha)

    
    dr= DataFrameConstructor(r, steps, test_size,model, alpha, mse_train, mse_test)
    df= df.append(dr) 


    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)




    # plot the figure!
    #plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)


trainVSmse(steps, sizes, mse_train_list, mse_test_list)  

In [ ]:

# display the data frame    
display(df)

### $r=3.1$, $steps= 50$
comment: at $r=3.1$ the population fluctuates, returning to the starting point every other year instead of every year. This is called period 2.

In [ ]:
r =3.1
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:
DatasetPlotter1(r, steps, dataset_X, dataset_y)

In [ ]:

sizes= (0.1,0.2,0.3, 0.5)
#alphas= np.arange(0.00001, 2, 0.01)
alpha=0.01
#kernel= ExpSineSquared(length_scale=1, periodicity=2)#TBD
#kernel=DotProduct() + WhiteKernel() 
kernel=  1.0 * Matern(length_scale=1.0, nu=1.5)
#kernel= 1.0 * RBF(1.0)
#kernel= 'linear'
#kernel= ExpSineSquared(length_scale=1, periodicity=2)#TBD

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 


for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)

    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test, kernel,alpha)
    
   
    dr= DataFrameConstructor(r, steps, test_size,model, alpha, mse_train, mse_test)
    df= df.append(dr) 
    
    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)    
    
    # plot the figure!
    
    
    plotter1(X_train, X_test, y_train, y_test,pred_train,pred_test)
    
    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
    zoomplotter_steps(1,10,5e-1,8e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)


trainVSmse(steps, sizes, mse_train_list, mse_test_list)  

In [ ]:
# display the data frame    

#display(df)

r =3.1
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:


sizes= (0.1,0.2,0.3, 0.5)
#alphas= np.arange(0.00001, 2, 0.01)
alpha=0.01

#kernel=DotProduct() + WhiteKernel() 
#kernel=  1.0 * Matern(length_scale=1.0, nu=1.5)
#kernel= 1.0 * RBF(1.0)
#kernel= 'linear'
kernel= ExpSineSquared(length_scale=1, periodicity=2)#TBD

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 


for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)

    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    
    dr= DataFrameConstructor(r, steps, test_size,model, alpha, mse_train, mse_test)
    df= df.append(dr) 
    
    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)  
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test,pred_train,pred_test)
    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    zoomplotter_steps(1,10,5e-1,9e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)



trainVSmse(steps, sizes, mse_train_list, mse_test_list)  
    

In [ ]:
# display the data frame    
#display(df)

r =3.1
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:

sizes= (0.1,0.2,0.3, 0.5)
alphas= np.arange(0.00001, 2, 0.01)

#kernel= ExpSineSquared(length_scale=1, periodicity=2)#TBD
#kernel=DotProduct() + WhiteKernel() 
#kernel=  1.0 * Matern(length_scale=1.0, nu=1.5)
#kernel= 1.0 * RBF(1.0)
#kernel= 'linear'
kernel= ExpSineSquared(length_scale=1, periodicity=2)#TBD




for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)


    #define
    mape_train_list=[]
    mape_test_list=[] 
    mse_train_list=[]
    mse_test_list=[] 
    
    #Select an appropriate value for alpha
    for alpha in alphas:
        print('alpha has value: %.3f' % alpha)
        print('test size :',test_size)
      

        mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test, kernel,alpha)
    
    
        # dr= DataFrameConstructor(r, steps, test_size,model, mse_train, mse_test)
        # df= df.append(dr) 
    

        # # comment to avoid a too long output
        # # plot the figure!
        # plotter(X_train, X_test, y_train, y_test, pred_train, pred_test)
        
        # plotter_steps(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

        #save errors in arrays
        mape_train_list.append(mape_train)
        mape_test_list.append(mape_test)
        
        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)
    
    alphaVSmse(alphas, mse_train_list, mse_test_list)    


In [ ]:
# # display the data frame    
# display(df1)
# display(df)

### r= 3.5
at $r=3.5$, the trajectory is period 4, as it takes 4 iterations for the population to return to its original position

In [ ]:
r=3.5
steps=50
dataset, dataset_y= Dataset_generator(r, steps)
DatasetPlotter1(r, steps, dataset, dataset_y)
flat_dataset_X = np.array([elem[1] for elem in dataset])[:,None]

In [ ]:

alphas= np.arange(0.00001, 1, 0.01)

#kernel= ExpSineSquared(length_scale=1, periodicity=4)#TBD
#kernel=DotProduct() + WhiteKernel() 
kernel=  1.0 * Matern(length_scale=1.0, nu=1.5)
#kernel= 1.0 * RBF(1.0)
#kernel= 'linear'
sizes= (0.1,0.2,0.3, 0.5)
for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    #define
    mape_train_list=[]
    mape_test_list=[] 
    mse_train_list=[]
    mse_test_list=[] 

    #Select an appropriate value for alpha
    for alpha in alphas:
        print('alpha has value: %.3f' % alpha)
    
        mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test, kernel,alpha)
        
        # dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
        # df= df.append(dr)
            
        #save errors in arrays
        mape_train_list.append(mape_train)
        mape_test_list.append(mape_test)
        
        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)  

        # # comment to avoid too long output
        # # plot the figure!
        # plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


        # plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
      
    
    
    
    alphaVSmse(alphas, mse_train_list, mse_test_list)  

In [ ]:
# display the data frame    
display(df)

r=3.5
steps=50
dataset, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[1] for elem in dataset])[:,None]

In [ ]:

alphas= np.arange(0.001, 1, 0.01)

kernel= ExpSineSquared(length_scale=1, periodicity=4)#TBD
#kernel=DotProduct() + WhiteKernel() 
#kernel=  1.0 * Matern(length_scale=1.0, nu=1.5)
#kernel= 1.0 * RBF(1.0)
#kernel= 'linear'
sizes= (0.1,0.2,0.3, 0.5)



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    #define
    mape_train_list=[]
    mape_test_list=[] 
    mse_train_list=[]
    mse_test_list=[] 
    #Select an appropriate value for alpha
    for alpha in alphas:
        print('alpha has value: %.3f' % alpha)
    
        mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test, kernel,alpha)
        
        # # plot the figure!
        # plotter(X_train, X_test, y_train, y_test, pred_train, pred_test)


        # plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
        #save errors in arrays
        mape_train_list.append(mape_train)
        mape_test_list.append(mape_test)
        
        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)  
    
    alphaVSmse(alphas, mse_train_list, mse_test_list)     
    
  

In [ ]:
# # display the data frame    
display(df)

r=3.5
steps=50
dataset, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[1] for elem in dataset])[:,None]

In [ ]:

alphas= np.arange(0.001, 1, 0.01)
kernel= ExpSineSquared(length_scale=1, periodicity=2)

sizes= (0.1,0.2,0.3, 0.5)


for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    #define
    mape_train_list=[]
    mape_test_list=[] 
    mse_train_list=[]
    mse_test_list=[] 

    #Select an appropriate value for alpha
    for alpha in alphas:
        print('alpha has value: %.3f' % alpha)
    
        mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test,kernel, alpha)
        # dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
        # df= df.append(dr)
            


        # # plot the figure!
        # plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


        # plotter_steps1(steps,r,test_size,X_train, X_test, y_train, y_test, pred_train, pred_test)

    
        #save errors in arrays
        mape_train_list.append(mape_train)
        mape_test_list.append(mape_test)

        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)  
    
    alphaVSmse(alphas, mse_train_list, mse_test_list) 



In [ ]:
# # display the data frame    

display(df)

r=3.5
steps=50
dataset, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[1] for elem in dataset])[:,None]

In [ ]:

alphas= np.arange(0.001, 1, 0.01)
#kernel= fixed in the gauss model #TBD

sizes= (0.1,0.2,0.3, 0.5)




for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
     
    #define
    mape_train_list=[]
    mape_test_list=[] 
    mse_train_list=[]
    mse_test_list=[] 
    #Select an appropriate value for alpha
    for alpha in alphas:
        print('alpha has value: %.3f' % alpha)
    
        mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = GAUS( X_train, X_test, y_train, y_test, alpha)

    
        # dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
        # df= df.append(dr)

        # # plot the figure!
        # plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


        # plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
        #save errors in arrays
        mape_train_list.append(mape_train)
        mape_test_list.append(mape_test)
        
        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)
    
      
    alphaVSmse(alphas, mse_train_list, mse_test_list) 

In [ ]:
# # display the data frame    

# display(df)

In [ ]:
alpha= 0.1
kernel= ExpSineSquared(length_scale=1, periodicity=4)
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    #Select an appropriate value for alpha
    #for alpha in arange(0.00001, 1, 0.01):
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,8e-1,3e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)  

# Aperiodic trajectories in the logistic map
For $r=3.7$, the (prime) period is longer than the iterations plotted and is actually infinite, and therefore the system is called aperiodic. To restate, this means that previous values of the logistic equation are never revisited for an aperiodic $r$ value. The ensuing plot has points that look random but are deterministic. The formation of aperiodic behavior from a deterministic system is termed mathematical chaos.

### Remark: 
As demonstrated by Lorenz in his pioneering work on flow, nonlinear dissipative systems capable of aperiodic behavior are extremely sensitive to initial conditions such that long-range behavior is impossible to predict.

In [ ]:
r =3.7
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
DatasetPlotter1(r, steps, dataset_X, dataset_y)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:
# alpha= 0.1
# kernel= RationalQuadratic(length_scale=1.0, alpha=1.5) #TBD

In [ ]:

alpha= 0.1
kernel= RationalQuadratic(length_scale=1.0, alpha=1.5) #TBD
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
  
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)


    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)


    zoomplotter_steps(1,10,9e-1,3e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)    

r =3.7
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:


alpha= 0.1
kernel= RationalQuadratic(length_scale=0.3, alpha=0.1) #TBD
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)

    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = GAUS(X_train, X_test, y_train, y_test, alpha)
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)
    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    zoomplotter_steps(1,10,9e-1,3e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)


trainVSmse(steps, sizes, mse_train_list, mse_test_list)

r =3.7
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:

alpha= 0.1
kernel= RationalQuadratic(length_scale=0.4, alpha=0.1) #TBD 
#kernel= RationalQuadratic(length_scale=1.0, alpha=5.5) #TBD
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
      
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)
    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    zoomplotter_steps(1,10,9e-1,3e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)


trainVSmse(steps, sizes, mse_train_list, mse_test_list)

r =3.7
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:

alpha= 0.1
#kernel= RationalQuadratic(length_scale=1.0, alpha=1.5) #TBD
#kernel=DotProduct() + WhiteKernel() 
#kernel=  1.0 * Matern(length_scale=1.0, nu=1.5)
kernel= 1.1 * RBF(0.5)
#kernel= 'linear'
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)
    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    zoomplotter_steps(1,10,9e-1,3e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

trainVSmse(steps, sizes, mse_train_list, mse_test_list)

r =3.7
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:

alpha= 0.1
kernel= ExpSineSquared(length_scale=1, periodicity=4)#TBD
#kernel=DotProduct() + WhiteKernel() 
#kernel=  1.0 * Matern(length_scale=1.0, nu=1.5)
#kernel= 1.0 * RBF(1.0)
#kernel= 'linear'

sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)
    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    zoomplotter_steps(1,10,9e-1,3e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)


trainVSmse(steps, sizes, mse_train_list, mse_test_list)

r =3.7
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:

alpha= 0.1
#kernel= ExpSineSquared(length_scale=1, periodicity=4)#TBD
#kernel=DotProduct() + WhiteKernel() 
kernel=  1.1 * Matern(length_scale=0.5, nu=1.5)
#kernel= 1.0 * RBF(1.0)
#kernel= 'linear'

sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)

    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
   
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)
    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    zoomplotter_steps(1,10,8e-1,3e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)


trainVSmse(steps, sizes, mse_train_list, mse_test_list)

r=3.7
steps=50
dataset, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[1] for elem in dataset])[:,None]

In [ ]:

alphas= np.arange(0.00001, 1, 0.01)
kernel= RationalQuadratic(length_scale=0.4, alpha=0.1)  #TBD
#kernel=  1.1 * Matern(length_scale=0.5, nu=1.5)
sizes= (0.1,0.2,0.3, 0.5)



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
     
    #define
    mape_train_list=[]
    mape_test_list=[] 
    mse_train_list=[]
    mse_test_list=[] 


    #Select an appropriate value for alpha
    for alpha in alphas:
        print('alpha has value', alpha)
    
        mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test,kernel, alpha)

        # dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
        # df= df.append(dr)

        # # plot the figure!
        # plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


        # plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
        #save errors in arrays
        mape_train_list.append(mape_train)
        mape_test_list.append(mape_test)
        
        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)    
    
    
    alphaVSmse(alphas, mse_train_list, mse_test_list) 

In [ ]:
# display the data frame    
# display(df)

r=3.7
steps=50
dataset, dataset_y= Dataset_generator(r, steps)
flat_dataset_X = np.array([elem[1] for elem in dataset])[:,None]

In [ ]:

alphas= np.arange(0.001, 1, 0.01)
#kernel= RationalQuadratic(length_scale=0.4, alpha=0.1)  #TBD
kernel=  1.1 * Matern(length_scale=0.5, nu=1.5)
sizes= (0.1,0.2,0.3, 0.5)



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
     
    #define
    mape_train_list=[]
    mape_test_list=[] 
    mse_train_list=[]
    mse_test_list=[] 

    #Select an appropriate value for alpha
    for alpha in alphas:
        print('alpha has value: %.3f' % alpha)
    
        mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test,kernel, alpha)
        
        #save errors in arrays
        mape_train_list.append(mape_train)
        mape_test_list.append(mape_test)
    
            
        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)  

        # dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
        # df= df.append(dr)
        # # plot the figure!
        # plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


        # plotter_steps1(steps,r,test_size,X_train, X_test, y_train, y_test, pred_train, pred_test)
    
    
    
    alphaVSmse(alphas, mse_train_list, mse_test_list) 

In [ ]:
# # display the data frame    

# display(df)

# r=3.7, steps= 200

In [ ]:
r =3.7
steps=200
dataset_X, dataset_y= Dataset_generator(r, steps)
DatasetPlotter1(r, steps, dataset_X, dataset_y)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:

alpha= 0.1
kernel= RationalQuadratic(length_scale=0.4, alpha=0.1)  
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,9e-1,3e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)    

In [ ]:

alpha= 0.1
#kernel= RationalQuadratic(length_scale=0.4, alpha=0.1)  
kernel= 1.1 * RBF(0.5)
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)

    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps1(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,9e-1,3e-1,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)    

### r=4

In [ ]:
r =4
steps=50
dataset_X, dataset_y= Dataset_generator(r, steps)
DatasetPlotter2(r, steps, dataset_X, dataset_y)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:
alpha= 0.1
kernel= RationalQuadratic(length_scale=0.1, alpha=0.1)  
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-3,1e-0,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)    

In [ ]:
kernel=  1.0 * Matern(length_scale=1.0, nu=0.5)
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-3,1e-0,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)    

In [ ]:
kernel=  6.0 * Matern(length_scale=1.0, nu=0.5)
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-3,1e-0,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)    

In [ ]:

alphas= np.arange(0.001, 1, 0.01)
#kernel= RationalQuadratic(length_scale=0.4, alpha=0.1)  #TBD
kernel=  6 * Matern(length_scale=0.5, nu=0.5)
sizes= (0.1,0.2,0.3, 0.5)



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
     
    #define
    mape_train_list=[]
    mape_test_list=[] 
    mse_train_list=[]
    mse_test_list=[] 

    #Select an appropriate value for alpha
    for alpha in alphas:
        print('alpha has value: %.3f' % alpha)
    
        mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE( X_train, X_test, y_train, y_test,kernel, alpha)
        
        #save errors in arrays
        mape_train_list.append(mape_train)
        mape_test_list.append(mape_test)
    
            
        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)  

        # dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
        # df= df.append(dr)
        # # plot the figure!
        # plotter1(X_train, X_test, y_train, y_test, pred_train, pred_test)


        # plotter_steps1(steps,r,test_size,X_train, X_test, y_train, y_test, pred_train, pred_test)
    
    
    
    alphaVSmse(alphas, mse_train_list, mse_test_list) 

In [ ]:
r =4
steps=200
dataset_X, dataset_y= Dataset_generator(r, steps)
DatasetPlotter2(r, steps, dataset_X, dataset_y)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:
kernel=  2.0 * Matern(length_scale=1.0, nu=0.5)
sizes= (0.1,0.2,0.3, 0.5)

#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-2,1e-0,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)   

In [ ]:
kernel=  2.0 * Matern(length_scale=0.5, nu=0.5)
sizes= (0.1,0.2,0.3, 0.5)
alpha=0.1
#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-2,1e-0,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)   

In [ ]:
kernel=  4.0 * Matern(length_scale=0.5, nu=0.5)
sizes= (0.1,0.2,0.3, 0.5)
alpha=0.1
#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-2,1e-0,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)   

In [ ]:
kernel=  6.0 * Matern(length_scale=0.5, nu=0.5)
sizes= (0.1,0.2,0.3, 0.5)
alpha=0.1
#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-2,1e-0,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)   

In [ ]:
#print(df)

In [ ]:
r =4
steps=1000
dataset_X, dataset_y= Dataset_generator(r, steps)
DatasetPlotter2(r, steps, dataset_X, dataset_y)
flat_dataset_X = np.array([elem[0] for elem in dataset_X])[:,None]

In [ ]:
kernel=  6.0 * Matern(length_scale=0.5, nu=0.5)
sizes= (0.1,0.2,0.3, 0.5)
alpha=0.1
#define
mape_train_list=[]
mape_test_list=[] 
mse_train_list=[]
mse_test_list=[] 



for test_size in sizes:
    print('test size :',test_size)
    X_train, X_test, y_train, y_test= split(flat_dataset_X, dataset_y,test_size)
    
    mape_train, mape_test, mse_train, mse_test, model, pred_train, pred_test = RIDGE(X_train, X_test, y_train, y_test, kernel,alpha)
    
    
    
    dr= DataFrameConstructor(r, steps, test_size, model,alpha, mse_train, mse_test)
    df= df.append(dr)

    #save errors in arrays
    mape_train_list.append(mape_train)
    mape_test_list.append(mape_test)
    
    mse_train_list.append(mse_train)
    mse_test_list.append(mse_test)
    
    # plot the figure!
    plotter2(X_train, X_test, y_train, y_test, pred_train, pred_test)


    plotter_steps2(steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)

    zoomplotter_steps(1,10,1e-2,1e-0,steps,r,test_size, X_train, X_test, y_train, y_test, pred_train, pred_test)
    
trainVSmse(steps, sizes, mse_train_list, mse_test_list)   

Uncommenting the following lines, we download the dataframe of the result, after convering it in a csv file.

In [ ]:
# df.to_csv('Experiments.csv',index=False)
# from google.colab import files
# files.download('Experiments.csv')

This work is attached to the Thesis document (Learning dynamical systems with kernels). 

In the thesis, theoretical explanations as well as comments on the results are presented.